You can read the paper for more detail. <BR>
Reference: https://ieeexplore.ieee.org/abstract/document/5197422

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
class Matrix_Factorization:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.0005):
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam
        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie
                  3. Save the training set.
                  4. Initialize P and Q matrix, with normal distribution with mean = 0. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
            
        
        """
        num_user = max(map(lambda t:t[0], train)) + 1
        num_movie =  max(map(lambda t:t[1], train)) + 1
        
        self.train = train
        
        self.P = np.random.normal(loc = 0, size=[num_user, self.num_of_latent])
        self.Q = np.random.normal(loc = 0, size=[num_movie, self.num_of_latent])
        
        rmse_lst = []
       
        
        """
            TODO: 5: Calculate the error, using P and Q matrix. 
                  6: We need to check if the absolute value error is less than some constant. Store the previous Q and P for adaptive learning rate.
                      If it is less than that constant then we update P and Q matrix. 
                      (When update, update the P and Q at the same time. Think about why it is important.)
                      Otherwise use the error to update the Q and P matrix.
                      
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """
 
        for f in range(self.iterations):
            ### Random Shuffle. Why is this called?
            np.random.shuffle(self.train)
            
            temp_mse = 0

            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            
            Count = 0
   
            for tup in self.train:
                u,i,rating = tup
                error = rating - self.Q[i].T @ self.P[u]
                
                if abs(error) > 20 :
                    continue
                Count += 1
                temp_mse += error**2
                
                # #### Don't Modify this code, helpful for converge.
                if np.isinf(self.Q).any() or np.isinf(self.P).any() or np.isnan(self.Q).any() or np.isnan(self.P).any():
                    pass
                else:
                    temp_Q = self.Q[i].copy()
                    self.Q[i] += 2 * self.alpha * error * self.P[u] - self.lam * self.Q[i]
                    self.P[u] += 2 * self.alpha * error * temp_Q - self.lam * self.P[u]
                
            rmse_lst.append((temp_mse / Count)**0.5)
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
            if len(rmse_lst)==1 or rmse_lst[-1] <rmse_lst [-2]:
                self.alpha = self.alpha * 1.2
            else:
                self.alpha = self.alpha * 0.8
                if (rmse_lst[-1] - rmse_lst[-2]) / rmse_lst[-2] > 0.1:
                    self.Q = previous_Q
                    self.P = previous_P
                    
            
        self.rmse = rmse_lst
        
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i, r = tup
        
        return self.Q[i].T @ self.P[u]
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(self.ind_predict(i))
        return res

In [6]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings.sort_values('timestamp').reset_index(drop = True).astype(int)
a = np.sort(ratings.movieId.unique())
index_dict = {}
for i in range(len(a)):
    index_dict[a[i]]  = i
ratings.movieId = ratings.movieId.apply(lambda x: index_dict[x])

grouped_userId = list(ratings.groupby(['userId']).apply(lambda x: x[['movieId','rating']].values))
grouped_movieId = list(ratings.groupby(['movieId']).apply(lambda x: x[['userId','rating']].values))

# New Section

In [7]:
np.random.seed(0)
train = set()
test = set()

for i in range(len(grouped_movieId)):
    temp_len = len(grouped_movieId[i])
    for j in range(temp_len):
        if j < temp_len * 0.8:
            train.add((grouped_movieId[i][j][0]-1,i - 1, grouped_movieId[i][j][1]))
        else:
            test.add((grouped_movieId[i][j][0] -1, i - 1,grouped_movieId[i][j][1]))
train = list(train)
test = list(test)

#### Try to understand what is done for data cleaning.

#### If you are doing things correctly (with tuning), you will get something lower than 1.1 test RMSE.

In [9]:
clf = Matrix_Factorization(alpha = 0.0005, iterations = 50, num_of_latent = 3, lam = 0.005)
clf.fit(train)

In [10]:
error = 0
pred = clf.predict(test)
print(pred)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

[2.673159701209974, 3.0436798343465123, 3.547708470033585, 2.4896214055123274, 1.7338196744160184, 2.7581237019532274, 3.8671637159016092, 3.6722766800372626, 2.794802269730087, 3.206067900302415, 3.1952522672758974, 2.3129701732161827, 3.297898133730694, 1.1793678986704763, 3.316128443975329, 3.3436797923284876, 2.2978477708495673, 2.290346072048403, 2.5462978341375804, 3.0880019108130807, 3.7357376561781654, 3.1911957242153774, 3.596102699175939, 4.416024343376769, 3.322265919921632, 3.9022730973079094, 4.017562013520197, 3.5934501409656625, 3.7016944329012165, 3.0158779419442467, 3.0165856887908205, 2.82706449151959, 3.963193453391358, 2.3095643699383746, 3.520578821474687, 3.413720994137506, 1.6957542893461344, 2.608989906650177, 3.245770823860465, 2.5952498435282636, 1.8749475258216988, 5.107549144494168, 2.5830261226492173, 3.374673990584589, 2.877240381864987, 4.214294407025619, 2.903266298343183, 3.293589002453685, 3.5556176743968546, 3.8009805442799594, 2.8335282477140242, 2.3

In [11]:
(error/ len(test))** (0.5)

1.0111550104492315

(185, 506, 5)


In [12]:
class Matrix_Factorization_with_bias:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.01):
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
 
            
            TODO: 1. Initialize all variables needed.
        """
            
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam


        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie.
                  3. Save the training set.
                  4. Initialize bu , bi and b. b is the global mean of the rating.
                  5. Initialize P and Q matrix. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
                  
            
        
        """
        num_user = max(map(lambda t:t[0], train)) + 1
        num_movie =  max(map(lambda t:t[1], train)) + 1
        
        self.train = train
        
        self.P = np.random.normal(loc = 0, size=[num_user, self.num_of_latent])
        self.Q = np.random.normal(loc = 0, size=[num_movie, self.num_of_latent])
        
        self.bu = np.random.uniform(size=num_user)
        self.bi = np.random.uniform(size=num_movie)
        self.b = np.mean([r[2] for r in train])
        
        rmse_lst = []
        self.score = []
        """
            TODO: 5: Calculate the error, using P , Q , bu , bi and b. 
                  6: Update the P , Q , bu , bi and b with error you calculate. 
                    (Think about why we don't need to check the absolute of error)
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """


        for f in range(self.iterations):
            
            np.random.shuffle(self.train)
            
            temp_mse = 0
            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            previous_bu = self.bu.copy()
            previous_bi = self.bi.copy()
            
            Count = 0
            for tup in self.train:
                u,i,rating = tup
                
                error = rating - self.Q[i].T @ self.P[u] - self.bu[u] - self.bi[i] - self.b
                


                Count += 1   
                temp_mse += error ** 2
                temp_Q = self.Q[i].copy()

                # You might want to update column by column
                self.Q[i] += 2 * self.alpha * error * self.P[u] - self.lam * self.Q[i]
                self.P[u] += 2 * self.alpha * error * temp_Q - self.lam * self.P[u]
                
                self.bu[u] += 2 * self.alpha * error - self.lam * self.bu[u]
                self.bi[i] += 2 * self.alpha * error - self.lam * self.bi[i]
            
            rmse_lst.append((temp_mse / Count)**0.5)
                
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
            
            if len(rmse_lst)==1 or rmse_lst[-1] < rmse_lst [-2]:
                self.alpha = self.alpha * 1.15
            else:
                self.alpha = self.alpha * 0.75
                if (rmse_lst[-1] - rmse_lst[-2]) / rmse_lst[-2] > 0.1:
                    self.Q = previous_Q
                    self.P = previous_P 
                    self.bu = previous_bu
                    self.bi = previous_bi

      
        self.rmse = rmse_lst
            
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i, r = tup
        
        return self.Q[i].T @ self.P[u] + self.bu[u] + self.bi[i] + self.b
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(self.ind_predict(i))
        return res

### If you are doing thing correctly (With tuning), you should get a Test RMSE below 1.

In [13]:
clf = Matrix_Factorization_with_bias(alpha = 0.0005, iterations = 50, num_of_latent = 3, lam = 0.005)
clf.fit(train)

In [14]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [15]:
(error/ len(test))** (0.5)

0.9504267701781014